In [1]:
import os
import cv2
import imageio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from tqdm import tqdm
from skimage import draw
from scipy import ndimage

### Starting points

In [2]:
# approximation
UPPER_BOUNDS_CAM1 = [[110,150], [95,175],
                     [90,200], [85,225], [82,250], [80,275], 
                     [78,300], [75,325], [73,350], [73,375],
                     [72,400], [71,425], [71,450], [71,475],
                     [70,500], [70,525], [70,550], [70,575],
                     [70,600], [70,625], [70,650], [70,675],
                     [70,700], [70,725], [70,750], [70,775],
                     [75,800], [73,825], [75,850], [76,875],
                     [77,900], [78,925], [78,950], [81,975],
                     [82,1000], [88,1025], [94,1050], 
                     [100,1075], [100,1100], [120,1125]]

# Starting points for boundary line
UPPER_BOUNDS_CAM3 = [[845,150], [845,175],
                     [840,200], [835,225], [827,250], [819,275], 
                     [805,300], [790,325], [775,350], [750,375],
                     [730,400], [710,425], [690,450], [665,475],
                     [630,500], [600,525], [560,550], [550,575],
                     [505,600], [400,625], [380,650], [380,675],
                     [380,700], [380,725], [380,750], [380,775],
                     [380,800], [380,825], [380,850], [380,875],
                     [380,900], [380,925], [380,950], [380,975],
                     [380,1000], [380,1025], [380,1050], [380,1075], 
                     [380,1100]]

# Cam3A details:

# 15 deg (leftmost)
UPPER_BOUNDS_CAM3A1 = [[870, 375]]

# 25 deg (mid)
UPPER_BOUNDS_CAM3A2 = [[885, 450]]

# 35 deg (rightmost)
UPPER_BOUNDS_CAM3A3 = [[845, 525]]


UPPER_BOUNDS_CAM3B = [[800,600], [800,625], [800,650], [800,675],
                     [800,700], [800,725], [800,750], [800,775],
                     [800,800], [800,825], [800,850], [800,875],
                     [800,900], [800,925], [800,950], [800,975],
                     [800,1000]]

LOWER_BOUNDS_CAM3C = [[700,900], [715,925], [725,950], [730,975],
                     [740,1000], [750,1025], [760,1050], [765,1075], 
                     [765,1100]]

UPPER_BOUNDS_CAM1 = np.array(UPPER_BOUNDS_CAM1)
UPPER_BOUNDS_CAM3 = np.array(UPPER_BOUNDS_CAM3)
UPPER_BOUNDS_CAM3A1 = np.array(UPPER_BOUNDS_CAM3A1)
UPPER_BOUNDS_CAM3A2 = np.array(UPPER_BOUNDS_CAM3A2)
UPPER_BOUNDS_CAM3A3 = np.array(UPPER_BOUNDS_CAM3A3)
UPPER_BOUNDS_CAM3B = np.array(UPPER_BOUNDS_CAM3B)
LOWER_BOUNDS_CAM3C = np.array(LOWER_BOUNDS_CAM3C)

INDICES_CAM1 = [2,6,10,14,18,22,26,30,34]
INDICES_CAM3A1 = [0]
INDICES_CAM3A2 = [0]
INDICES_CAM3A3 = [0]
INDICES_CAM3B = [2,6,10,14]
INDICES_CAM3C = [2,6]

In [3]:
# filename is an image file
def preprocess_image(filename):
    bgr_image       = cv2.imread(filename)
    bgr_image       = increase_brightness(bgr_image, value=40)
    
    rgb_image       = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    rgb_image45     = ndimage.rotate(rgb_image, -45)
    rgb_image35     = ndimage.rotate(rgb_image, -35)
    rgb_image25     = ndimage.rotate(rgb_image, -25)
    rgb_image15     = ndimage.rotate(rgb_image, -15)
    
    gray_image      = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    gray_image45    = ndimage.rotate(gray_image, -45)
    gray_image35    = ndimage.rotate(gray_image, -35)
    gray_image25    = ndimage.rotate(gray_image, -25)
    gray_image15    = ndimage.rotate(gray_image, -15)
    
    _, binary_image = cv2.threshold(gray_image, 220, 255, cv2.THRESH_BINARY)
    binary_image    = binary_image/255
    
    _, binary_image45 = cv2.threshold(gray_image45, 220, 255, cv2.THRESH_BINARY)
    binary_image45    = binary_image45/255
    
    # A3 (rightmost)
    _, binary_image35 = cv2.threshold(gray_image35, 220, 255, cv2.THRESH_BINARY)
    binary_image35    = binary_image35/255
    
    # A2 (mid)
    _, binary_image25 = cv2.threshold(gray_image25, 220, 255, cv2.THRESH_BINARY)
    binary_image25    = binary_image25/255
    
    # A1 (leftmost)
    _, binary_image15 = cv2.threshold(gray_image15, 220, 255, cv2.THRESH_BINARY)
    binary_image15    = binary_image15/255
    
    return (binary_image, rgb_image, binary_image45, rgb_image45, 
            binary_image35, rgb_image35, binary_image25, rgb_image25, 
            binary_image15, rgb_image15)

In [4]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [5]:
# points is a starting coordinate --> [y,x]
# this function is used in inversed_hieghts() function
def count_pixel(binary_image, points, topdown=True):
    y = points[0]
    x = points[1]
    no_of_pixels = 0
    
    while binary_image[y,x] == 1:
        if topdown == True:
            y += 1
        else:
            y -= 1
        no_of_pixels += 1
    
    return no_of_pixels

In [6]:
# calculate_heights was previously inversed_heights()
def calculate_heights(binary_image, starting_points, topdown=True):
    heights = []
    for points in starting_points:
        height = count_pixel(binary_image, points, topdown)
        heights.append(height)
    
    return np.array(heights)

In [7]:
# calculates the actual water height
# (container_height - inversed_height)
def actual_height(inversed_height):
    return 320-inversed_height

In [8]:
def pixel_to_mm(value):
    return value*0.828

In [9]:
# determine the actual boundary location
def find_boundary_coords(inversed_height_points, upper_bounds):
    y_locs = upper_bounds[:,0] + inversed_height_points
    x_locs = upper_bounds[:,1]
    
    y_locs = y_locs[:,np.newaxis]
    x_locs = x_locs[:,np.newaxis]
    
    coords = np.hstack((y_locs, x_locs))
    
    return coords

In [10]:
def line_indices(boundary_coords):
    rrs = []
    ccs = []
    for i in range(len(boundary_coords)-1):
        y0 = boundary_coords[i][0]
        x0 = boundary_coords[i][1]
        y1 = boundary_coords[i+1][0]
        x1 = boundary_coords[i+1][1]
        
        rr, cc, _ = draw.line_aa(y0, x0, y1, x1)
        rrs.append(rr)
        ccs.append(cc)
    
    return rrs, ccs

In [11]:
def draw_lines(rgb_image, rrs, ccs):
    for rr, cc in zip(rrs, ccs):
        rgb_image[rr,cc,0] = 0
        rgb_image[rr,cc,1] = 0
        rgb_image[rr,cc,2] = 255
        
        rgb_image[rr+1,cc+1,0] = 0
        rgb_image[rr+1,cc+1,1] = 0
        rgb_image[rr+1,cc+1,2] = 255
    
    return rgb_image

In [12]:
# Keep in mind that camera a and 3b is using topdown approach
def take_data(heights, indices):
    heights_to_save = heights[indices]
    heights_to_save = actual_height(heights_to_save)
    heights_to_save = pixel_to_mm(heights_to_save)
    return heights_to_save

In [13]:
# Camera 3c is using bottom up approach
def take_data_cam_3c(heights, indices=INDICES_CAM3C):
    heights_to_save = heights[indices]
    heights_to_save = pixel_to_mm(heights_to_save)
    return heights_to_save

In [14]:
'''
INDICES_CAM1 = [2,6,10,14,18,22,26,30,34]
INDICES_CAM3A1 = [0]
INDICES_CAM3A2 = [0]
INDICES_CAM3A3 = [0]
INDICES_CAM3B = [2,6,10,14]
INDICES_CAM3C = [2,6]
'''

def calculate_all_sections(filename):
    binary_image, rgb_image, binary_image45, rgb_image45, binary_image35, rgb_image35, binary_image25, rgb_image25, binary_image15, rgb_image15 = preprocess_image(filename)
    
    heights_3c = calculate_heights(binary_image, LOWER_BOUNDS_CAM3C, topdown=False)
    heights_3c = take_data_cam_3c(heights_3c, INDICES_CAM3C)
    
    heights_3b = calculate_heights(binary_image45, UPPER_BOUNDS_CAM3B, topdown=True)
    heights_3b = take_data(heights_3b, INDICES_CAM3B)
    
    # A3 (rightmost, 35 deg)
    heights_3a3 = calculate_heights(binary_image35, UPPER_BOUNDS_CAM3A3, topdown=True)
    heights_3a3 = take_data(heights_3a3, INDICES_CAM3A3)
    
    # A2 (mid, 25 deg)
    heights_3a2 = calculate_heights(binary_image25, UPPER_BOUNDS_CAM3A2, topdown=True)
    heights_3a2 = take_data(heights_3a2, INDICES_CAM3A2)
    
    # A1 (leftmost, 15 deg)
    heights_3a1 = calculate_heights(binary_image15, UPPER_BOUNDS_CAM3A1, topdown=True)
    heights_3a1 = take_data(heights_3a1, INDICES_CAM3A1)
    
    return heights_3a1, heights_3a2, heights_3a3, heights_3b, heights_3c

### Main program goes below (use this)

In [15]:
# HOW TO USE THIS CODE
# declare empty list satu2. misal heights_3a1 = [], dst
# for satu_gambar in semua_gambar_cam_3:
#     calculate_all_sections()
#     append satu2
# list = np.array(list)
# lalu pakai fungsi concatenate_heights()

PATH = 'cam3_images/'
filenames = os.listdir(PATH)

heights_3a1s = []
heights_3a2s = []
heights_3a3s = []
heights_3bs = []
heights_3cs = []

for filename in tqdm(filenames):
    file_path = PATH + filename
    
    heights_3a1, heights_3a2, heights_3a3, heights_3b, heights_3c = calculate_all_sections(file_path)
    
    heights_3a1s.append(heights_3a1)
    heights_3a2s.append(heights_3a2)
    heights_3a3s.append(heights_3a3)
    heights_3bs.append(heights_3b)
    heights_3cs.append(heights_3c)
    
heights_3a1s = np.array(heights_3a1s)
heights_3a2s = np.array(heights_3a2s)
heights_3a3s = np.array(heights_3a3s)
heights_3bs = np.array(heights_3bs)
heights_3cs = np.array(heights_3cs)

100%|██████████| 4/4 [00:22<00:00,  5.73s/it]


In [16]:
all_heights = np.hstack((heights_3a1s,heights_3a2s,heights_3a3s,heights_3bs,heights_3cs))

In [17]:
all_heights.shape

(4, 9)

In [18]:
column_names = ['Point 0', 'Point 1', 'Point 2', 'Point 3', 'Point 4', 
                'Point 5', 'Point 6', 'Point 7', 'Point 8']
df = pd.DataFrame(all_heights, columns=column_names)

In [19]:
df

,Point 0,Point 1,Point 2,Point 3,Point 4,Point 5,Point 6,Point 7,Point 8
0,21.528,38.916,37.260,31.464,33.120,26.496,41.400,78.660,117.576
1,21.528,38.088,34.776,29.808,24.012,26.496,41.400,78.660,117.576
2,22.356,39.744,41.400,29.808,24.012,30.636,42.228,78.660,117.576
3,22.356,38.088,42.228,28.980,24.012,29.808,49.680,79.488,117.576


### Main program goes below (test)

In [20]:
filename = 'cam3_images/CCFL040_10_2000_335_up_a_0000.bmp'

In [21]:
binary_image, rgb_image, binary_image45, rgb_image45, binary_image35, rgb_image35, binary_image25, rgb_image25, binary_image15, rgb_image15 = preprocess_image(filename)

In [22]:
heights_3c = calculate_heights(binary_image, LOWER_BOUNDS_CAM3C, topdown=False)
heights_3c = take_data_cam_3c(heights_3c)
heights_3c

array([ 79.488, 117.576])

In [23]:
heights_3b = calculate_heights(binary_image45, UPPER_BOUNDS_CAM3B, topdown=True)
heights_3b = take_data(heights_3b)

TypeError: take_data() missing 1 required positional argument: 'indices'

In [24]:
heights_3b

array([294, 293, 285, 284, 280, 269, 291, 291, 278, 281, 284, 280, 254,
       246, 260, 266, 266])

In [25]:
# A3 (rightmost, 35 deg)
heights_3a3 = calculate_heights(binary_image35, UPPER_BOUNDS_CAM3A3, topdown=True)
heights_3a3 = take_data(heights_3a3)

TypeError: take_data() missing 1 required positional argument: 'indices'

In [26]:
heights_3a3

array([269])

In [27]:
# A2 (mid, 25 deg)
heights_3a2 = calculate_heights(binary_image25, UPPER_BOUNDS_CAM3A2, topdown=True)
heights_3a2 = take_data(heights_3a2)

TypeError: take_data() missing 1 required positional argument: 'indices'

In [28]:
heights_3a2

array([274])

In [29]:
# A1 (leftmost, 15 deg)
heights_3a1 = calculate_heights(binary_image15, UPPER_BOUNDS_CAM3A1, topdown=True)
heights_3a1 = take_data(heights_3a1)

TypeError: take_data() missing 1 required positional argument: 'indices'

In [30]:
heights_3a1

array([293])